IMPORTS

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import ast
import re

FONCTIONS

In [2]:
limite_liste = 2000
apparition_realisateur = 8
apparition_acteur = 10
apparition_production = 5
len_production = 3

In [275]:
def debut2(titre):
    # titre = titre.lower()
    # titre = titre.replace(' ', '').replace('.', '').replace(',', '').replace(':', '').replace('&', '').replace('?', '').replace('!', '')
    # titre = titre.replace('(', '').replace(')', '').replace('[', '').replace(']', '').replace('{', '').replace('}', '').replace('/', '')
    # titre_final = ''
    # for n in range(len(titre)):
    #     if titre[n].isalpha() or titre[n].isnumeric():
    #         titre_final += titre[n]
    #     else:
    #         break

    # titre_final = titre_final[0:8]

    titre = titre.lower().replace("'", '').replace('(', '').replace(')', '').replace('[', '').replace(']', '').replace('{', '').replace('}', '').replace('/', '').replace(' ', '').replace('.', '').replace(',', '').replace('&', '').replace('?', '').replace('!', '')
    titre_final = ''
    if ':' in titre:
        for n in range(len(titre)):
            if titre[n] != ':':
                if titre[n].isalpha() or titre[n].isnumeric():
                    titre_final += titre[n]   
            else:
                break           

    return titre_final

In [4]:
def name(liste_films, df):

    df = df.drop(['ordering', 'category', 'job', 'characters'], axis = 1)
    df['nconst'] = df['nconst'] + ', '
    df = df.groupby('tconst').sum()
    df = df.reset_index()

    df = pd.merge(liste_films,
                  df,
                  left_on = 'film_id_out_KNN',
                  right_on = 'tconst',
                  how = 'left')

    df = df.drop('film_id_out_KNN', axis = 1)
    df['nconst'] = df['nconst'].astype(str)
    df['nconst'] = df['nconst'].apply(lambda x : x.split(','))
    df = df.explode('nconst')
    df['tconst'] = df['tconst'] + ','
    df = df.groupby('nconst').sum()
    df = df.reset_index()
    df = df[~(df['nconst']== ' ')]
    df['nconst'] = df['nconst'].astype(str)
    df['tconst'] = df['tconst'].astype(str)
    df['nconst'] = df['nconst'].apply(lambda x : x.replace(' ', ''))
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', ''))

    return df

In [5]:
def acteur_name(df):

    df['tconst_y'] = df['tconst_y'].astype(str)
    df['tconst'] = df['tconst_x'] + df['tconst_y']
    df = df[['nconst', 'primaryName', 'tconst']]
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", ''))
    df['tconst'] = df['tconst'].apply(lambda x : x.split(','))
    df = df.rename({'tconst': 'knownForTitles'}, axis = 1)
    df = df.rename({'nconst': 'acteur'}, axis = 1)

    return df

In [6]:
def realisateur_name(df):
    
    df = df.groupby('nconst').sum()
    df = df.reset_index()
    df = pd.merge(df,
                  df_title_crew,
                  on = 'nconst',
                  how = 'left')

    df = df[~(df['nconst'] == 'nan')]
    df['tconst_x'] = df['tconst_x'].astype(str)
    df['tconst_y'] = df['tconst_y'].astype(str)
    df['tconst'] = df['tconst_x'] + ',' + df['tconst_y']
    df = df.drop(['tconst_y', 'tconst_x'], axis = 1)
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", '').replace(",,", ','))
    df['tconst'] = df['tconst'].astype(str)
    df['tconst'] = df['tconst'].apply(lambda x : x.split(','))
    df['tconst'] = df['tconst'].apply(lambda x : set(x))
    df['tconst'] = df['tconst'].apply(lambda x : list(x))

    df = pd.merge(df,
                  df_name_basics,
                  on = 'nconst',
                  how = 'left')

    df['tconst_x'] = df['tconst_x'].astype(str)
    df['tconst_y'] = df['tconst_y'].astype(str)
    df['tconst'] = df['tconst_x'] + ',' + df['tconst_y']
    df = df.drop(['tconst_y', 'tconst_x'], axis = 1)
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", '').replace(",,", ','))
    df['tconst'] = df['tconst'].astype(str)
    df['tconst'] = df['tconst'].apply(lambda x : x.split(','))
    df['tconst'] = df['tconst'].apply(lambda x : set(x))
    df['tconst'] = df['tconst'].apply(lambda x : list(x))

    df = df.rename({'tconst': 'knownForTitles'}, axis = 1)
    df = df.rename({'nconst': 'realisateurs'}, axis = 1)
    df = df.explode('knownForTitles')
    df = df[~(df['knownForTitles'] == '')]

    return df

In [7]:
def transfo_liste(x):
  if isinstance(x, str):
    return ast.literal_eval(x)
  else:
    return x

In [8]:
def debut(titre):
    titre = titre.lower().replace(' ', '')
    titre_final = ''
    for n in range(len(titre)):
        if titre[n].isalpha():
            titre_final += titre[n]
    return titre_final[0:20]

In [9]:
def len_pandas(x, colonne):
    x_escaped = re.escape(x)
    return df_films[df_films[f'{colonne}'].fillna('').str.contains(x_escaped, na=False, regex=True)].shape[0]

In [10]:
def list_colonnes(df_films, colonne):

    # def debut2(titre):
    #     titre = titre.lower().replace(' ', '')
    #     titre_final = ''
    #     for n in range(len(titre)):
    #         if titre[n].isalpha() or titre[n].isnumeric():
    #             titre_final += titre[n]
    #         else:
    #             break
    #     if len(titre_final, colonne) > 20:
    #         titre_final = titre_final[0:20]
    #     return titre_final

    # if colonne in ['acteur_out_KNN', 'realisateurs_out_KNN', 'title_2', 'production_companies_name_out_KNN', 'genre_out_KNN']:
    df_films[colonne] = df_films[colonne].astype(str)

    # if type(df_films[colonne].iloc[0]) == str:
    #     if colonne == 'title_debut':
    #         df_films[f'{colonne}2'] = df_films['title'].apply(lambda x: x.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(" ", '').lower())
    #         df_films[f'{colonne}'] = df_films[f'{colonne}2'].apply(debut2) 
    #         df_films[f'{colonne}'] = df_films[f'{colonne}'].apply(lambda x : x.split(' '))
    #     else:
    df_films[f'{colonne}2'] = df_films[colonne].apply(lambda x: x.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(" ", '').lower())
    df_films[f'{colonne}'] = df_films[f'{colonne}2'].apply(lambda x : x.split(','))

    for n in range(len(df_films)):
        for element in df_films[colonne].iloc[n]:
              
            if element == "":
                df_films[colonne].iloc[n].remove(element)
            elif element == "nan":
                df_films[colonne].iloc[n].remove(element)

In [11]:
def liste_recurrence(df_films, colonne):

    df_films[colonne] = df_films[colonne].astype(str)
    df_films[f'{colonne}2'] = df_films[colonne].apply(lambda x: x.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(" ", '').lower())
    df_films[f'{colonne}'] = df_films[f'{colonne}2'].apply(lambda x : x.split(','))

    liste_production_companies_name_out_KNN= []

    for genres in df_films[colonne] :
        for element in genres:
            liste_production_companies_name_out_KNN.append(element)

    set_production_companies_name_out_KNN = set(liste_production_companies_name_out_KNN)

    if '' in set_production_companies_name_out_KNN:
        set_production_companies_name_out_KNN.remove('')

    dico_production_companies_name_out_KNN = {}

    for element in set_production_companies_name_out_KNN:
        if colonne == 'production_companies_name_out_KNN':
            if liste_production_companies_name_out_KNN.count(element) > apparition_production:
                if len(element) > len_production:
                    dico_production_companies_name_out_KNN.update({element : liste_production_companies_name_out_KNN.count(element)})
        else:
            dico_production_companies_name_out_KNN.update({element : liste_production_companies_name_out_KNN.count(element)})

    sorted_dico_production_companies_name_out_KNN = dict(sorted(dico_production_companies_name_out_KNN.items(), key=lambda item:item[1], reverse = True))
    
    limite_realisateurs = 200
    limite_acteur = 400
    limite_production_companies_name = 150
    limite_genre = len(set_production_companies_name_out_KNN)

    if colonne == 'acteur_out_KNN':
        limite = limite_acteur
    elif colonne == 'realisateurs_out_KNN':
        limite = limite_realisateurs
    elif colonne == 'production_companies_name_out_KNN':
        limite = limite_production_companies_name
    elif colonne == 'genre_out_KNN':
        limite = limite_genre

    liste_totale_production_companies_name_out_KNN = list(sorted_dico_production_companies_name_out_KNN.keys())

    if len(liste_totale_production_companies_name_out_KNN) > limite_liste:
        liste_totale_production_companies_name_out_KNN = liste_totale_production_companies_name_out_KNN[:limite_liste]

    
    liste_limitee_production_companies_name_out_KNN = liste_totale_production_companies_name_out_KNN[:limite]
    
    if 'nan' in set_production_companies_name_out_KNN:
        set_production_companies_name_out_KNN.remove('nan')
    if "" in set_production_companies_name_out_KNN:
        set_production_companies_name_out_KNN.remove("")

    return liste_totale_production_companies_name_out_KNN, liste_limitee_production_companies_name_out_KNN

In [12]:
def classement_element(df_films, colonne, liste_totale, dico_total_final):

  if colonne in ['acteur_out_KNN', 'realisateurs_out_KNN']:
    df_films[f'trie_{colonne}'] = ''
    df_films[f'name_trie_{colonne}'] = ''

    for element in liste_totale:
      df_films[f'trie_{colonne}'] = df_films.apply(lambda row : (row[f'trie_{colonne}'] + f', {element}') if element in row[colonne] else row[f'trie_{colonne}'], axis = 1)
      df_films[f'name_trie_{colonne}'] = df_films.apply(lambda row : (row[f'name_trie_{colonne}'] + f', {dico_total_final[element]}') if element in row[colonne] else row[f'name_trie_{colonne}'], axis = 1)
    
    df_films[f'trie_{colonne}'] = df_films[f'trie_{colonne}'].apply(lambda x : x[2:])
    df_films[f'name_trie_{colonne}'] = df_films[f'name_trie_{colonne}'].apply(lambda x : x[2:])

    df_films[f'trie_{colonne}'] = df_films[f'trie_{colonne}'].apply(lambda x : x.split(', '))
    df_films[f'name_trie_{colonne}'] = df_films[f'name_trie_{colonne}'].apply(lambda x : x.split(', '))

  else:
    df_films[f'trie_{colonne}'] = ''

    for element in liste_totale:
      df_films[f'trie_{colonne}'] = df_films.apply(lambda row : (row[f'trie_{colonne}'] + f', {element}') if element in row[colonne] else row[f'trie_{colonne}'], axis = 1)

    df_films[f'trie_{colonne}'] = df_films[f'trie_{colonne}'].apply(lambda x : x[2:])

    df_films[f'trie_{colonne}'] = df_films[f'trie_{colonne}'].apply(lambda x : x.split(', '))

In [13]:
# def liste_recurrence(colonne):
#     tous_les_genres = set()

#     for genres in df_films[colonne] :
#         tous_les_genres.update(genres)
#     tous_les_genres = list(tous_les_genres)

#     if "" in tous_les_genres:
#         tous_les_genres.remove("")

#     if colonne in colonnes_limite.keys():

#         last_list = []
#         for element in tous_les_genres:
#             if len(element) >= colonnes_limite[colonne][0]:
#                 last_list.append(element)
#         tous_les_genres = last_list

#     if colonne == 'production_companies_name_out_KNN':
#         df_films['production_companies_name_Marvel'] = df_films['production_companies_name_out_KNN2'].apply(lambda x: 'marvel' in x)
#         df_films['production_companies_name_Disney'] = df_films['production_companies_name_out_KNN2'].apply(lambda x: 'disney' in x)

#     dict_real = {}

#     for element in tous_les_genres:
#         dict_real.update({element : len_pandas(element, colonne)})

#     sorted_dict_real = dict(sorted(dict_real.items(), key=lambda item:item[1], reverse = True))

#     if 'nan' in sorted_dict_real.keys():
#         sorted_dict_real.pop('nan')

#     list_real = list(sorted_dict_real.keys())

#     return list_real

In [14]:
def bool_colonnes(df_films, liste, colonne):

    for genre in liste:
        df_films[f'{colonne}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)

    # df_films = df_films.drop(f'{colonne}2', axis = 1)
    # df_films = df_films.drop(colonne, axis = 1)
    df_films = df_films.rename({colonne: f'{colonne}_out_KNN'})

In [15]:
def transfo_col_bool(df_films):

    df_films_not_bool = df_films.select_dtypes(exclude = 'bool')
    df_films_bool = df_films.select_dtypes(include = 'bool')
    df_films_bool = df_films_bool.astype(str)
    df_films_bool = df_films_bool.replace('True', '1').replace('False', '0')
    df_films_bool = df_films_bool.astype(int)
    df_films = pd.concat([df_films_not_bool, df_films_bool], axis = 1)
    df_films_not_bool, df_films_bool = 0, 0

    return df_films

BASES ET MERGES

In [16]:
df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/3595914245.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])


In [17]:
df_title_ratings = pd.read_csv('BD/title.ratings.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])

In [18]:
df1 = pd.merge(df_title_basics,
               df_title_ratings,
               how = 'outer',
               on = 'tconst')

In [19]:
df_title_ratings, df_title_basics = 0, 0

In [20]:
df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/2474832937.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])


In [21]:
df_3 = pd.merge(df1,
                df_title_akas,
                how = 'outer',
                left_on = 'tconst',
                right_on = 'titleId')

In [22]:
# ON GARDE QUE LES FILMS

df_3 = df_3[(df_3['titleType'] == 'movie')]

In [23]:
df1, df_title_akas = 0, 0

In [24]:
df_tmbd_full = pd.read_csv('BD/tmdb_full.csv', na_values= ['\\N'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/2845617658.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmbd_full = pd.read_csv('BD/tmdb_full.csv', na_values= ['\\N'])


In [25]:
df_films = pd.merge(df_tmbd_full,
                    df_3,
                    how = 'outer',
                    left_on = 'imdb_id',
                    right_on = 'tconst')

In [26]:
df_tmbd_full, df_3 = 0, 0

COLONNES NUMERIQUES

In [27]:
# On remplit les cellules vides par 0

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].fillna(0)

In [28]:
# On remplace les '\\N' par 0

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].replace('\\N', 0)

In [29]:
# On met ces colonnes au format numérique

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].apply(pd.to_numeric)

In [30]:
# On transforme les types de données pour 'release_date'

df_films['release_date'] = pd.to_datetime(df_films['release_date'])
df_films['release_date'] = df_films['release_date'].replace('\\N', 0)
df_films['release_date'] = df_films['release_date'].dt.year

In [31]:
# On transforme les types de données pour 'startYear'

df_films['startYear'] = df_films['startYear'].replace('\\N', 0)
df_films['startYear'] = pd.to_numeric(df_films['startYear'], downcast="integer")

In [32]:
# On crée la colonne year_exact

df_films['year_exact'] = df_films[['release_date', 'startYear']].max(axis = 1)

In [33]:
# On crée la colonne 'Decennie'

df_films['Decennie'] = ''
df_films['year_exact'] = df_films['year_exact'].fillna('0')
df_films['year_exact'] = df_films['year_exact'].replace('nan', '0')
df_films['year_exact'] = df_films['year_exact'].astype(str)

In [34]:
df_films['Decennie'] = df_films['year_exact'].str[:3] + "0"
df_films['year_exact'] = pd.to_numeric(df_films['year_exact'])
df_films['Decennie'] = pd.to_numeric(df_films['Decennie'])

RUNTIME

In [35]:
# On met de côté les valeurs nulles

df_null1 = df_films[((df_films['runtimeMinutes'].isna()) & (df_films['runtime'].isna()))]
df_null2 = df_films[((df_films['runtimeMinutes'] == 0) & (df_films['runtime'] == 0))]
df_null3 = df_films[((df_films['runtimeMinutes'] == 0) & (df_films['runtime'].isna()))]
df_null4 = df_films[((df_films['runtimeMinutes'].isna()) & (df_films['runtime'] == 0))]

In [36]:
df_null = pd.concat([df_null1, df_null2, df_null3, df_null4])

In [37]:
df_null1, df_null2, df_null3, df_null4 = 0, 0, 0, 0

In [38]:
# On prend les valeurs utiles

df_utile = df_films[~((df_films['runtimeMinutes'].isna()) & (df_films['runtime'].isna()))]
df_utile = df_utile[~((df_utile['runtimeMinutes'] == 0) & (df_utile['runtime'] == 0))]
df_utile = df_utile[~((df_utile['runtimeMinutes'] == 0) & (df_utile['runtime'].isna()))]
df_utile = df_utile[~((df_utile['runtimeMinutes'].isna()) & (df_utile['runtime'] == 0))]

In [39]:
# Calcul différence de runtime des 2 tables

df_utile['difference_runtime'] = abs(df_utile['runtime'] - df_utile['runtimeMinutes'])

In [40]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_utile[(df_utile['runtimeMinutes'] == df_utile['runtime'])]

In [41]:
# Création colonne RuntimeExact
df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']

df_a_travailler_1 = df_utile[~(df_utile['runtimeMinutes'] == df_utile['runtime'])]

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/3570380868.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']


In [42]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]

In [43]:
# On conserve la donnée existante

df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/3521993277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)


In [44]:
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]
df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/1261991755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])


In [45]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_runtime'] <= 20]

In [46]:
# On prend comme base la valeur max des 2 colonnes

df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/880983650.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [47]:
df_a_travailler_3 = df_a_travailler_2[~(df_a_travailler_2['difference_runtime'] <= 20)]

In [48]:
# Pour le restant des données, on conserve la valeur max des 2 colonnes

df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/264539281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [49]:
df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])

In [50]:
df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0
df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4 = 0, 0, 0, 0

In [51]:
df_films = pd.concat([df_final, df_null])

In [52]:
df_final, df_null = 0, 0

VOTES

In [53]:
# On met de côté les valeurs inutiles

df_null5 = df_films[((df_films['vote_average'].isna()) & (df_films['averageRating'].isna()))]
df_null6 = df_films[((df_films['vote_average'] == 0) & (df_films['averageRating'] == 0))]
df_null7 = df_films[((df_films['vote_average'] == 0) & (df_films['averageRating'].isna()))]
df_null8 = df_films[((df_films['vote_average'].isna()) & (df_films['averageRating'] == 0))]

In [54]:
df_null9 = pd.concat([df_null5, df_null6, df_null7, df_null8])

In [55]:
df_null5, df_null6, df_null7, df_null8 = 0, 0, 0, 0

In [56]:
# On prend les valeurs utiles

df_ratings = df_films[~((df_films['vote_average'].isna()) & (df_films['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'] == 0))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'].isna()) & (df_ratings['averageRating'] == 0))]

In [57]:
df_ratings['vote_average'] = round(df_ratings['vote_average'], 1)

In [58]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_ratings[df_ratings['averageRating'] == df_ratings['vote_average']]
df_a_travailler_1 = df_ratings[~(df_ratings['averageRating'] == df_ratings['vote_average'])]

In [59]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[(df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0)]
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0))]

In [60]:
df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/2546417221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])


In [61]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] <= 1]
df_a_travailler_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] > 1]

In [62]:
df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
df_a_travailler_3['vote_exact'] = round(df_a_travailler_3['vote_exact'], 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/4062215657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/4062215657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travai

In [63]:
# On crée la colonne vote_exact pour garder la moyenne des votes

df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_3['vote_exact'] = df_a_garder_3[['averageRating', 'vote_average']].mean(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/3653439724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/3653439724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/36534

In [64]:
# On concatène tous les tableaux conservés

df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])
df_final['arrondi_vote_exact'] = round(df_final['vote_exact'], 0)

In [65]:
df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0
df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4 = 0, 0, 0, 0

In [66]:
df_films = pd.concat([df_final, df_null9])

In [67]:
df_final, df_null9 = 0, 0

GENRE

In [68]:
df_films = df_films.rename(columns = {'genres_x' : 'genres_y', 'genres_y' : 'genres_x'})

In [69]:
df_films[df_films[['genres_y', 'genres_x']] == '\\N'] = df_films[df_films[['genres_y', 'genres_x']] == '\\N'].replace('\\N', np.nan)

In [70]:
df_films['genres_y'] = df_films['genres_y'].apply(transfo_liste)

In [71]:
df_films['genres_y'] = df_films['genres_y'].apply(lambda x: x if (type(x) == list) else [])

In [72]:
df_films['genres_x'] = df_films['genres_x'].str.replace(",", ", ")

In [73]:
df_films['genres_x_list'] = df_films['genres_x'].fillna('').str.split(', ')

In [74]:
df_films['genre'] = df_films.apply(lambda row: list(set(row['genres_y'] + row['genres_x_list'])),axis=1)

In [75]:
df_films['genre'] = df_films['genre'].apply(lambda x: x if x != [] else np.nan)

TITLE

In [76]:
df_films['title'] = df_films['title_y'].fillna(df_films['title_x']).fillna(df_films['primaryTitle']).fillna(df_films['original_title']).fillna(df_films['originalTitle'])

ADULT

In [77]:
df_films['adult'] = (df_films['isAdult'] == 1) | df_films['adult']

TCONST

In [78]:
df_films['film_id'] = df_films['imdb_id'].fillna(df_films['tconst']).fillna(df_films['titleId'])

VOTE COUNT

In [79]:
df_films['vote_count_mean'] = (df_films['vote_count'] + df_films['numVotes'])/2

FINAL LANGUAGE

In [80]:
colonnes_y = ['production_countries', 'spoken_languages', 'production_companies_country']

for cl in colonnes_y:
 
    df_films[cl] = df_films[cl].apply(transfo_liste)
    df_films[cl] = df_films[cl].apply(lambda x: x if (type(x) == list) else [])

In [81]:
df_films['original_language_x_list'] = df_films['original_language'].fillna('').str.split(', ')

In [82]:
df_films['language_x_list'] = df_films['language'].fillna('').str.split(', ')

In [83]:
df_films['final_language'] = df_films.apply(lambda row: list(set(row['production_countries'] + row['spoken_languages'] + row['production_companies_country'] + row['original_language_x_list'] + row['language_x_list'])),axis=1)

In [84]:
df_films['final_language'] = df_films['final_language'].apply(lambda x: x if x != [] else np.nan)

In [85]:
df_films['final_language'] = df_films['final_language'].apply(lambda x: ', '.join(x) if type(x) == list else x)

In [86]:
df_films = df_films[(df_films['final_language'].isna() == True) | (df_films['final_language'] == '') | (df_films['final_language'].str.contains('fr') == True) | (df_films['final_language'].str.contains('FR') == True)  | (df_films['final_language'].str.contains('us') == True) | (df_films['final_language'].str.contains('US') == True)]

CLEAN FINAL DU MERGE ET GROUP BY

In [87]:
df_str = df_films.select_dtypes(exclude = 'number')

In [88]:
df_str['language_fr'] = df_str['final_language'].apply(lambda x : 1 if ('fr' in x or 'FR' in x) else 0)

In [89]:
df_str = df_str.sort_values(by = 'language_fr', ascending = False)

In [90]:
df_str_drop_dup = df_str.drop_duplicates(subset = ['film_id'], keep = 'first')

In [91]:
df_num = pd.concat([df_films['film_id'], df_films.select_dtypes(include = 'number')], axis = 1)

In [92]:
df_num = df_num.drop(['revenue', 'budget'], axis = 1)

In [93]:
df_revenue_budget = df_films[['film_id', 'revenue', 'budget']]

In [94]:
df_revenue_budget_gb = df_revenue_budget.groupby('film_id').max()

In [95]:
df_num_gb = df_num.groupby('film_id').max()

In [96]:
df_num2 = pd.merge(df_num_gb,
                   df_revenue_budget_gb,
                   how = 'outer',
                   on = 'film_id')

In [97]:
df_films = pd.merge(df_str_drop_dup,
                   df_num2,
                   on = 'film_id',
                   how = 'left')

In [98]:
df_num2, df_str_drop_dup, df_num_gb, df_revenue_budget_gb, df_revenue_budget, df_num, df_str = 0, 0, 0, 0, 0, 0, 0

In [99]:
df_films = df_films[df_films['status'] == 'Released']

In [100]:
df_films = df_films[~(df_films['vote_count_mean'] == 0)] # A voir si on le conserve
df_films = df_films[~(df_films['year_exact'] == 0)]
df_films = df_films[~(df_films['runtime_exact'].isna() == True)]
df_films['region'] = df_films['region'].fillna('Unknown')

CHANGEMENT POUR LANGUE FR

In [101]:
df_films['prod_FR'] = df_films.apply(lambda row : 1 if 'FR' in row['production_companies_country'] else 0, axis = 1)

CHANGEMENT POUR FILM US

In [102]:
df_films['prod_US'] = df_films.apply(lambda row : 1 if 'US' in row['production_companies_country'] else 0, axis = 1)

In [103]:
df1, df_3, df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4, df_a_travailler_1, df_a_travailler_2, df_a_travailler_3, df_final, df_null = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [104]:
df_films[df_films.select_dtypes('number') == True] = df_films[df_films.select_dtypes('number') == True].fillna(0)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/3129310626.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_films[df_films.select_dtypes('number') == True] = df_films[df_films.select_dtypes('number') == True].fillna(0)


In [105]:
df_films[['vote_exact', 'arrondi_vote_exact']] = df_films[['vote_exact', 'arrondi_vote_exact']].astype(float)

In [106]:
df_films['titleType'] = df_films['titleType'].fillna('movie')

In [107]:
df_num = df_films.select_dtypes(include = 'number')
df_num = df_num.fillna(0)
df_str = df_films.select_dtypes(exclude = 'number')
df_str = df_str.fillna("Unknown")

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/3351577065.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_str = df_str.fillna("Unknown")


In [108]:
df_films = pd.concat([df_num, df_str], axis = 1)

In [109]:
df_str, df_num = 0, 0

In [110]:
df_films = df_films[(df_films['final_language'].str.contains('fr') == True) | (df_films['final_language'].str.contains('FR') == True)]

AJOUT TMDB FILMS

In [111]:
df_tmbd_full = pd.read_csv('BD/tmdb_full.csv', na_values= ['\\N'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/2845617658.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmbd_full = pd.read_csv('BD/tmdb_full.csv', na_values= ['\\N'])


In [112]:
for element in df_tmbd_full.columns:
    df_tmbd_full = df_tmbd_full.rename(columns = {element : f'{element}_tmdb'})

for element in df_films.columns:
    df_films = df_films.rename(columns = {element : f'{element}_df'})

In [113]:
df_essai_tmdb = pd.merge(df_films,
                         df_tmbd_full,
                         how = 'outer',
                         right_on = 'imdb_id_tmdb',
                         left_on = 'film_id_df')

In [114]:
df_essai_tmdb = df_essai_tmdb.drop(['language_fr_df', 'genres_y_df', 'genres_x_df', 'genres_x_list_df', 'original_title_df', 'title_x_df', 'titleType_df', 'titleId_df', 'title_y_df', 'original_title_tmdb', 'runtime_df', 'runtimeMinutes_df', 'difference_runtime_df', 'vote_average_df', 'difference_vote_df', 'vote_count_mean_df', 'vote_count_df', 'vote_count_mean_df'], axis = 1)

In [115]:
df_essai_tmdb['title_final'] = df_essai_tmdb['title_df'].fillna(df_essai_tmdb['title_tmdb'])
df_essai_tmdb = df_essai_tmdb.drop(['title_df', 'title_tmdb'], axis = 1)

In [116]:
df_essai_tmdb['film_id_final'] = df_essai_tmdb['film_id_df'].fillna(df_essai_tmdb['imdb_id_tmdb'])
df_essai_tmdb = df_essai_tmdb.drop(['film_id_df', 'imdb_id_tmdb'], axis = 1)

In [117]:
df_essai_tmdb = df_essai_tmdb.rename({'genre_df' : 'genre_final',
                                      'production_companies_name_df' : 'production_companies_name_final'}, axis = 1)

In [118]:
df_essai_tmdb = df_essai_tmdb.sort_values(by = 'genre_final', ascending = False)

In [119]:
df_essai_tmdb = df_essai_tmdb.drop_duplicates(subset = 'film_id_final', keep = 'first')

In [120]:
df_essai_tmdb = df_essai_tmdb.rename({'release_date_tmdb' : 'year_tmdb',
                                      'vote_average_tmdb' : 'vote_exact_tmdb'}, axis = 1)
df_essai_tmdb['vote_exact_tmdb'] = df_essai_tmdb['vote_exact_tmdb'].astype(float)

In [121]:
df_essai_tmdb['vote_exact_int_tmdb'] = ''
df_essai_tmdb['vote_exact_int_tmdb'] = df_essai_tmdb['vote_exact_tmdb'].apply(lambda x : round(x, 0))
df_essai_tmdb['arrondi_vote_exact_final'] = df_essai_tmdb['arrondi_vote_exact_df'].fillna(df_essai_tmdb['vote_exact_int_tmdb'])

df_essai_tmdb['arrondi_vote_exact_final'] = df_essai_tmdb['arrondi_vote_exact_final'].astype(float)

In [122]:

df_essai_tmdb['year_tmdb'] = pd.to_datetime(df_essai_tmdb['year_tmdb'])
df_essai_tmdb['year_tmdb'] = df_essai_tmdb['year_tmdb'].apply(lambda x : x.year)
df_essai_tmdb['year_tmdb'] = df_essai_tmdb['year_tmdb'].fillna(0)
df_essai_tmdb['year_tmdb'] = df_essai_tmdb['year_tmdb'].astype(int)

In [123]:
df_essai_tmdb['Decennie_tmdb'] = ''
df_essai_tmdb['year_tmdb'] = df_essai_tmdb['year_tmdb'].fillna('0')
df_essai_tmdb['year_tmdb'] = df_essai_tmdb['year_tmdb'].replace('nan', '0')
df_essai_tmdb['year_tmdb'] = df_essai_tmdb['year_tmdb'].astype(str)
df_essai_tmdb['Decennie_tmdb'] = df_essai_tmdb['year_tmdb'].str[:3] + "0"
df_essai_tmdb['year_tmdb'] = pd.to_numeric(df_essai_tmdb['year_tmdb'])
df_essai_tmdb['Decennie_tmdb'] = pd.to_numeric(df_essai_tmdb['Decennie_tmdb'])

In [130]:
df = pd.read_csv('BD/P2_G5_films.csv.gz', compression = 'gzip')
colonnes_df = list(df.columns)
colonnes_df_2 = []

for element in colonnes_df:
    if '_out_KNN' in element:
        colonnes_df_2.append(element[:-8])
    else:
        colonnes_df_2.append(element)

In [156]:
colonnes_df_2 = colonnes_df_2[:16]

In [132]:
if 'popularity' not in colonnes_df_2:
    colonnes_df_2.append('popularity')

if 'Decennie' not in colonnes_df_2:
    colonnes_df_2.append('Decennie')

if 'vote_exact_tmdb' not in colonnes_df_2:
    colonnes_df_2.append('vote_exact_tmdb')

In [136]:
for element in df_essai_tmdb.columns:
    if 'final' in element:
        df_essai_tmdb = df_essai_tmdb.rename({element : element.replace('final', 'df')}, axis = 1)

In [137]:
for element2 in colonnes_df_2:
    for element in df_essai_tmdb.columns:
        if 'final' not in element:
            if element2 in element and 'int' not in element and 'arrondi' not in element and 'count' not in element :
                if 'df' in element:
                    df_essai_tmdb = df_essai_tmdb.rename({element : f'{element2}_df'}, axis = 1)
                else:
                    df_essai_tmdb = df_essai_tmdb.rename({element : f'{element2}_tmdb'}, axis = 1)

In [142]:
for element in colonnes_df_2:
    for element2 in df_essai_tmdb.columns:
        if 'final' not in element:
            if element in element2:
                df_essai_tmdb[element2] = df_essai_tmdb[element2].astype(str)
                df_essai_tmdb[element2] = df_essai_tmdb[element2].apply(lambda x : x.replace('[', '').replace(']', '').replace("'", '').replace(",,", ',').replace('nan', ''))

In [143]:
for element2 in colonnes_df_2:
    for element in df_essai_tmdb.columns:
        if 'final' not in element:
            if element2 in element and 'int' not in element and 'arrondi' not in element and 'count' not in element :
                if element2 in element :
                    if f'{element2}_df' in df_essai_tmdb.columns and f'{element2}_tmdb' in df_essai_tmdb.columns:
                        df_essai_tmdb[f'{element2}_final'] = df_essai_tmdb[f'{element2}_df'].fillna(df_essai_tmdb[f'{element2}_tmdb'])
                        df_essai_tmdb = df_essai_tmdb.drop(f'{element2}_df', axis = 1)
                        df_essai_tmdb = df_essai_tmdb.drop(f'{element2}_tmdb', axis = 1)
                    elif f'{element2}_df' in df_essai_tmdb.columns and f'{element2}_tmdb' not in df_essai_tmdb.columns:
                        df_essai_tmdb[f'{element2}_final'] = df_essai_tmdb[f'{element2}_df']
                        df_essai_tmdb = df_essai_tmdb.drop(f'{element2}_df', axis = 1)
                    elif f'{element2}_df' not in df_essai_tmdb.columns and f'{element2}_tmdb' in df_essai_tmdb.columns:
                        df_essai_tmdb[f'{element2}_final'] = df_essai_tmdb[f'{element2}_tmdb']
                        df_essai_tmdb = df_essai_tmdb.drop(f'{element2}_tmdb', axis = 1)

In [146]:
df_essai_tmdb['language_df'] = df_essai_tmdb['language_df'].fillna('Unknown')

In [147]:
df_unknown = df_essai_tmdb[df_essai_tmdb['language_df'] == 'Unknown']
df_known = df_essai_tmdb[~(df_essai_tmdb['language_df'] == 'Unknown')]

In [148]:
df_unknown = df_unknown[df_unknown['spoken_languages_tmdb'].str.contains('fr') | df_unknown['spoken_languages_tmdb'].str.contains('en')]

In [149]:
df_essai_tmdb = pd.concat([df_known, df_unknown])

In [150]:
colonnes = []
for element in df_essai_tmdb.columns:
    if element[-5:] == 'final':
        colonnes.append(element[:-6])
        df_essai_tmdb = df_essai_tmdb.rename({element : element[:-6]}, axis =1)

df_essai_tmdb = df_essai_tmdb[colonnes]

In [151]:
df_essai_tmdb = df_essai_tmdb.rename({'genre' : 'genre_out_KNN',
                                      'title' : 'title_out_KNN',
                                      'film_id': 'film_id_out_KNN',
                                      'production_companies_name' : 'production_companies_name_out_KNN'}, axis = 1)

In [155]:
df_films = df_essai_tmdb

In [164]:
df_films['popularity'] = df_films['popularity'].apply(lambda x : 0 if x == '' else x)
df_films['popularity'] = df_films['popularity'].apply(lambda x : float(x))

In [170]:
df_films['Decennie'] = df_films['Decennie'].apply(lambda x : 0 if x == '' else x)
df_films['Decennie'] = df_films['Decennie'].apply(lambda x : float(x))

In [172]:
df_films['vote_exact'] = df_films['vote_exact'].apply(lambda x : 0 if x == '' else x)
df_films['vote_exact'] = df_films['vote_exact'].apply(lambda x : float(x))

In [177]:
df_films['runtime_exact'] = df_films['runtime_exact'].apply(lambda x : 0 if x == '' else x)
df_films['runtime_exact'] = df_films['runtime_exact'].astype(float)


In [178]:
df_films = df_films[df_films['popularity'] >= 0]
df_films = df_films[df_films['Decennie'] > 1970]
df_films = df_films[df_films['vote_exact'] > 3]
df_films = df_films[df_films['runtime_exact'] > 0]
df_films = df_films[df_films['status'] == 'Released']
print(len(df_films))

35499


COLONNES ACTEUR ET REALISATEUR

In [179]:
liste_films = pd.DataFrame(df_films['film_id_out_KNN'])

In [180]:
liste_films = liste_films.drop_duplicates()

In [181]:
df_title_principals = pd.read_csv('BD/title.principals.tsv.gz', compression = 'gzip', sep = '\t')

In [182]:
df_acteurs = df_title_principals[(df_title_principals['category'] == 'actor') | (df_title_principals['category'] == 'actress') | (df_title_principals['category'] == 'self')]

In [183]:
df_realisateurs = df_title_principals[df_title_principals['category'] == 'director']

In [184]:
df_title_principals = 0

In [185]:
df_title_crew = pd.read_csv('BD/title.crew.tsv.gz', compression = 'gzip', sep = '\t')

In [186]:
df_title_crew = df_title_crew.drop('writers', axis = 1)

In [187]:
df_title_crew = pd.merge(liste_films,
                         df_title_crew,
                         left_on = 'film_id_out_KNN',
                         right_on = 'tconst',
                         how = 'left')

In [188]:
df_title_crew = df_title_crew.drop('film_id_out_KNN', axis = 1)

In [189]:
df_title_crew = df_title_crew.rename({'directors': 'nconst'}, axis = 1)

In [190]:
df_title_crew['nconst'] = df_title_crew['nconst'].astype(str)

In [191]:
df_title_crew['nconst'] = df_title_crew['nconst'].apply(lambda x : x.split(","))

In [192]:
df_title_crew = df_title_crew.explode('nconst')

In [193]:
df_title_crew = df_title_crew[~(df_title_crew['nconst'] == " ")]

In [194]:
df_title_crew['tconst'] = df_title_crew['tconst'] + ', '

In [195]:
df_title_crew = df_title_crew.groupby('nconst').sum()

In [196]:
df_title_crew = df_title_crew.reset_index()

In [197]:
df_name_basics = pd.read_csv('BD/name.basics.tsv.gz', compression = 'gzip', sep = '\t')

In [198]:
df_name_basics = df_name_basics.drop(['birthYear', 'deathYear', 'primaryProfession'], axis = 1)

In [199]:
df_name_basics = df_name_basics.rename({'knownForTitles' : 'tconst'}, axis = 1)

In [200]:
df_name_basics['tconst'] = df_name_basics['tconst'].apply(lambda x : x.split(','))

In [201]:
df_name_basics['nconst'] = df_name_basics['nconst'].astype(str)

In [202]:
df_name_basics['nconst'] = df_name_basics['nconst'].apply(lambda x : x.replace(' ', ''))

In [203]:
df_acteurs = name(liste_films, df_acteurs)

In [204]:
df_realisateurs = name(liste_films, df_realisateurs)

In [205]:
df_acteurs_name = pd.merge(df_acteurs,
                            df_name_basics,
                            on = 'nconst',
                            how = 'left')

In [206]:
df_acteurs = 0

In [207]:
df_acteurs_name = acteur_name(df_acteurs_name)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/3296483890.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", ''))
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/3296483890.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tconst'] = df['tconst'].apply(lambda x : x.split(','))


In [208]:
df_realisateurs_name = realisateur_name(df_realisateurs)

In [209]:
df_acteurs, df_realisateurs, df_name_basics, df_title_crew = 0, 0, 0, 0

In [210]:
df_realisateurs_name.to_csv('P2_G5_realisateurs.csv.gz', index=False, compression='gzip')
df_realisateurs = df_realisateurs_name

In [211]:
df_acteurs_name.to_csv('P2_G5_acteurs.csv.gz', index=False, compression='gzip')
df_acteurs = df_acteurs_name

In [212]:
df_acteurs = df_acteurs.rename({'knownForTitles' : 'films'}, axis = 1)

In [213]:
df_acteurs2 = df_acteurs.explode('films')

In [214]:
acteurs_repetitions = pd.DataFrame(df_acteurs2['acteur'].value_counts()).reset_index()

In [215]:
acteurs_repetitions = acteurs_repetitions[acteurs_repetitions['count'] > apparition_acteur]

In [216]:
acteurs_recurrents = pd.merge(acteurs_repetitions,
                              df_acteurs,
                              how = 'left',
                              on = 'acteur')

In [217]:
acteurs_recurrents2 = acteurs_recurrents.explode('films')

In [218]:
acteurs_recurrents2['films'] = acteurs_recurrents2['films'].str.replace("'","")
acteurs_recurrents2['acteur'] = acteurs_recurrents2['acteur'] + ', '

In [219]:
acteurs_recurrents2 = acteurs_recurrents2.groupby('films').sum()

acteurs_recurrents2['acteur'] = acteurs_recurrents2['acteur'].astype(str)

In [220]:
acteurs_recurrents2['acteur'] = acteurs_recurrents2['acteur'].apply(lambda x : x.replace(", , ", ''))

acteurs_recurrents2 = acteurs_recurrents2.reset_index()
acteurs_recurrents2 = acteurs_recurrents2[['films', 'acteur']]

In [221]:
df_films = pd.merge(df_films,
                    acteurs_recurrents2,
                    how = 'left',
                    left_on = 'film_id_out_KNN',
                    right_on = 'films')

In [222]:
acteurs_repetitions['acteur'] = acteurs_repetitions['acteur'].astype(str)

In [223]:
# DF REALISATEURS

realisateurs_repetitions = pd.DataFrame(df_realisateurs['realisateurs'].value_counts()).reset_index()

realisateurs_repetitions = realisateurs_repetitions[realisateurs_repetitions['count'] > apparition_realisateur]

realisateurs_repetitions2 = realisateurs_repetitions.groupby('realisateurs').sum()

realisateurs_repetitions2 = realisateurs_repetitions2.reset_index()

In [224]:
realisateurs_connus = pd.merge(realisateurs_repetitions2,
                               df_realisateurs,
                               on = 'realisateurs',
                               how = 'left')

In [225]:
realisateurs_connus = realisateurs_connus[['knownForTitles', 'realisateurs']]

realisateurs_connus['realisateurs'] = realisateurs_connus['realisateurs'] + ','

realisateurs_connus2 = realisateurs_connus.groupby('knownForTitles').sum()

In [226]:
df_films = pd.merge(df_films,
                    realisateurs_connus2,
                    how = 'left',
                    left_on = 'film_id_out_KNN',
                    right_on = 'knownForTitles')

df_realisateurs['realisateurs'] = df_realisateurs['realisateurs'].astype(str)

df_films['realisateurs'] = df_films['realisateurs'].astype(str)

In [191]:
# df_films['title_debut'] = df_films['title'].apply(debut)
# df_films['title_debut'] = df_films['title_debut'].astype(str)
# df['title_len'] = df['title'].apply(lambda row : len(row.replace(' ','')))

In [227]:
df_films = df_films.rename({'acteur' : 'acteur_out_KNN',
                            'realisateurs' : 'realisateurs_out_KNN'}, axis = 1)

SUPPRESSION DES TITLE SANS CYRILLIQUE

In [228]:
df_films = df_films[~df_films['title_out_KNN'].str.contains(r'[^\x00-\x7F]+', regex=True)]

LISTE DES ACTEURS ET REALISATEURS

In [229]:
df_realisateurs_name = pd.read_csv('BD/P2_G5_realisateurs.csv.gz', compression = 'gzip')
df_acteurs_name = pd.read_csv('BD/P2_G5_acteurs.csv.gz', compression = 'gzip')

In [230]:
df_realisateurs_name['knownForTitles'] = df_realisateurs_name['knownForTitles'] + ','
df_realisateurs_name['primaryName'] = df_realisateurs_name['primaryName'] + ','

df_realisateurs_name = df_realisateurs_name.groupby('realisateurs').sum()

In [231]:
df_realisateurs_name['primaryName'] = df_realisateurs_name['primaryName'].apply(lambda x : x.split(','))
df_realisateurs_name['primaryName'] = df_realisateurs_name['primaryName'].apply(lambda x : set(x))
df_realisateurs_name['primaryName'] = df_realisateurs_name['primaryName'].apply(lambda x : str(x))
df_realisateurs_name['primaryName'] = df_realisateurs_name['primaryName'].apply(lambda x : x.replace('{', '').replace('}', '').replace(',', '').replace('"', "").replace("[", '').replace("]", '').replace("'", ''))

In [232]:
df_realisateurs_name['knownForTitles'] = df_realisateurs_name['knownForTitles'].apply(lambda x : x.split(','))
df_realisateurs_name['knownForTitles'] = df_realisateurs_name['knownForTitles'].apply(lambda x : set(x))
df_realisateurs_name = df_realisateurs_name.reset_index()

In [233]:
df_acteurs_name['knownForTitles'] = df_acteurs_name['knownForTitles'].apply(lambda x : set(x))

In [234]:
dico_acteurs = {}

for n in range(len(df_acteurs_name)):
    dico_acteurs.update({df_acteurs_name['acteur'].iloc[n] : df_acteurs_name['primaryName'].iloc[n]})

In [235]:
dico_realisateurs = {}

for n in range(len(df_realisateurs_name)):
    dico_realisateurs.update({df_realisateurs_name['realisateurs'].iloc[n] : df_realisateurs_name['primaryName'].iloc[n][1:]})

In [236]:
dico_total = dico_acteurs | dico_realisateurs

In [237]:
dico_total_final = {}

for element in dico_total:
    if element not in dico_total_final.keys():
        dico_total_final.update({element : dico_total[element]})

In [238]:
df_acteurs_name, df_realisateurs_name = 0,0

In [239]:
dico_total, dico_acteurs, dico_realisateurs = 0,0,0

In [240]:
acteurs_recurrents, acteurs_recurrents2, acteurs_repetitions, realisateurs_repetitions2, realisateurs_repetitions, realisateurs_connus2, realisateurs_connus, df_tmbd_full, df_utile, df_realisateurs, df_ratings, df_essai_tmdb, df_acteurs2, df_acteurs, df = 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

COMPTE ACTEURS / REALISATEURS / PRODUCTION

In [241]:
len(df_films)

27363

In [242]:
df_films = df_films.rename({'genre_' : 'genre_out_KNN',
                            'production_companies_name_' : 'production_companies_name_out_KNN'}, axis = 1)

In [243]:
list_colonnes(df_films, 'genre_out_KNN')

In [244]:
list_colonnes(df_films, 'realisateurs_out_KNN')

In [245]:
list_colonnes(df_films, 'acteur_out_KNN')

In [246]:
list_colonnes(df_films, 'production_companies_name_out_KNN')

In [247]:
liste_totale_genre, liste_limitee_genre = liste_recurrence(df_films, 'genre_out_KNN')

In [248]:
liste_totale_realisateurs, liste_limitee_realisateurs = liste_recurrence(df_films, 'realisateurs_out_KNN')

In [249]:
liste_totale_acteur, liste_limitee_acteur = liste_recurrence(df_films, 'acteur_out_KNN')

In [250]:
liste_totale_production, liste_limitee_production = liste_recurrence(df_films, 'production_companies_name_out_KNN')

In [251]:
len(liste_totale_production), len(liste_totale_acteur), len(liste_totale_realisateurs), len(liste_totale_genre), 

(1703, 2000, 753, 27)

In [252]:
len(liste_limitee_production), len(liste_limitee_acteur), len(liste_limitee_realisateurs), len(liste_limitee_genre), 

(150, 400, 200, 27)

In [253]:
classement_element(df_films, 'genre_out_KNN', liste_totale_genre, dico_total_final)

In [254]:
classement_element(df_films, 'realisateurs_out_KNN', liste_totale_realisateurs, dico_total_final)

In [255]:
classement_element(df_films, 'acteur_out_KNN', liste_totale_acteur, dico_total_final)

In [256]:
classement_element(df_films, 'production_companies_name_out_KNN', liste_totale_production, dico_total_final)

CREATION COLONNES EN BOOL

In [257]:
bool_colonnes(df_films, liste_limitee_genre, 'trie_genre_out_KNN')

In [258]:
bool_colonnes(df_films, liste_limitee_realisateurs, 'trie_realisateurs_out_KNN')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/1408838262.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/1408838262.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/1408838262.py:4: PerformanceWarning: DataFram

In [259]:
bool_colonnes(df_films, liste_limitee_acteur, 'trie_acteur_out_KNN')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/1408838262.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/1408838262.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/1408838262.py:4: PerformanceWarning: DataFram

In [260]:
bool_colonnes(df_films, liste_limitee_production, 'trie_production_companies_name_out_KNN')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/1408838262.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/1408838262.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/1408838262.py:4: PerformanceWarning: DataFram

In [261]:
df_films['production_companies_name_Marvel'] = df_films['trie_production_companies_name_out_KNN'].apply(lambda x: 'marvel' in x)
df_films['production_companies_name_Disney'] = df_films['trie_production_companies_name_out_KNN'].apply(lambda x: 'disney' in x)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/195742928.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films['production_companies_name_Marvel'] = df_films['trie_production_companies_name_out_KNN'].apply(lambda x: 'marvel' in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/195742928.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films['production_companies_name_Disney'] = df_films['trie_production_companies_name_out_KNN'].apply(lambda x: 'disney' in x)


In [333]:
liste_titres = []

for n in range(len(df_films)):
    liste_titres.append(debut2(df_films['title_out_KNN'].iloc[n]))

dico_titres = {}
set_titres = set(liste_titres)

for element in set_titres:
    if len(element) > 3:
        if liste_titres.count(element) >=3:
            dico_titres.update({element : liste_titres.count(element)})

if '' in set_titres:
    set_titres.remove('')
    
sorted_dico_titres = dict(sorted(dico_titres.items(), key=lambda item:item[1], reverse = True))  

df_films['debut_titre_out_KNN'] = df_films['title_out_KNN'].apply(debut2)

df_films['debut_titre_critere_out_KNN'] = ''

df_films['debut_titre_critere_out_KNN'] = df_films.apply(lambda x : x['debut_titre_out_KNN'] if x['debut_titre_out_KNN'] in sorted_dico_titres.keys() else '', axis = 1)
df_films['debut_titre_critere_out_KNN'] = df_films['debut_titre_critere_out_KNN'].apply(lambda x : x.split())

liste_limitee_title = sorted_dico_titres.keys()

In [335]:
bool_colonnes(df_films, liste_limitee_title, 'debut_titre_critere_out_KNN')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/1408838262.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/1408838262.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_87604/1408838262.py:4: PerformanceWarning: DataFram

DROP DES COLONNES

In [337]:
colonnes = ['realisateurs_nan', 'acteur_nan', 'genre2', 'acteur2', 'realisateurs2', 'production_companies_name2', 'acteur_,', 'original_language', 'spoken_languages', 'language', 'original_language_x_list', 'language_x_list', 'films', 'genres_x_list', 'genres_x', 'genres_y', 'original_title', 'title_y', 'title_x', 'primaryTitle', 'originalTitle', 'isAdult', 'runtime', 'release_date', 'vote_average', 'numVotes', 'vote_count', 'startYear', 'endYear', 'runtimeMinutes', 'averageRating', 'difference_runtime', 'difference_vote', 'isOriginalTitle', 'revenue', 'budget', 'ordering', 'types', 'attributes', 'video', 'id', 'films', 'imdb_id', 'tconst', 'titleId', 'production_companies_country', 'acteur', 'realisateurs', 'realisateurs_a', 'realisateurs_7', 'realisateurs_0', 'realisateurs_4', 'realisateurs_9','realisateurs_5', 'realisateurs_8', 'realisateurs_m','realisateurs_3','realisateurs_1','realisateurs_,', 'realisateurs_n','realisateurs_6','realisateurs_2','acteur_a','acteur_7', 'acteur_0','acteur_9','acteur_5','acteur_8', 'acteur_m','acteur_4','acteur_3', 'acteur_1', 'acteur_', 'acteur_ ', 'acteur_n', 'acteur_6', 'acteur_2']

for element in colonnes:
    if element in df_films.columns:
        df_films = df_films.drop(element, axis = 1)

BOOL DES COLONNES

In [338]:
df_films = transfo_col_bool(df_films)

In [339]:
colonnes = list(df_films.select_dtypes(include = 'number').columns)
supprimer = []

df_null = pd.DataFrame(df_films[colonnes].sum()).transpose()

for element in colonnes:
    if df_null[element].iloc[0] == 0:
        supprimer.append(element)

if len(supprimer) > 0:
    df_films = df_films.drop(supprimer, axis = 1)

RENAME DES COLONNES

In [340]:
list_rename = ['production_companies_name_', 'genre_', 'status', 'production_companies_name', 'genre', 'production_countries', 'language_fr', 'title_debut', 'titleType', 'title_len', 'region', 'backdrop_path', 'homepage', 'overview', 'poster_path', 'film_id', 'title', 'final_language', 'tagline', 'title']

In [341]:
dico_rename = {}
for element in list_rename:
    if element in df_films.columns:
        dico_rename.update({element: f'{element}_out_KNN'})

In [342]:
df_films = df_films.rename(dico_rename, axis = 1)

FILLNA

In [343]:
df_films_str = df_films.select_dtypes(exclude = 'number')

In [344]:
df_films_num = df_films.select_dtypes(include = 'number')

In [345]:
df_films_str = df_films_str.fillna('Unknown')

In [346]:
df_films_num = df_films_num.fillna(0)

In [347]:
df_films = pd.concat([df_films_str, df_films_num], axis = 1)

In [348]:
rename = ['trie_genre_out_KNN_', 'trie_realisateurs_out_KNN_', 'trie_acteur_out_KNN_', 'trie_production_companies_name_out_KNN_']

In [349]:
for element in df_films.columns:
    for element2 in rename:
        if element2 in element:
            element3 = element.replace('_out_KNN', '').replace('trie_', '')
            df_films = df_films.rename({element : element3}, axis = 1)

In [361]:
for element in df_films.columns:
    if 'debut_titre_critere_out_KNN' in element:
        df_films = df_films.rename({element : element.replace('_out_KNN', '')}, axis = 1)

In [358]:
df_films['vote_exact_tmdb'] = df_films['vote_exact_tmdb'].apply(lambda x : float(x))
df_films['arrondi_vote_exact'] = df_films['vote_exact_tmdb'].apply(lambda x : round(x,0))

In [352]:
df_films['runtime_exact'] = df_films['runtime_exact'].replace('Unknown', 0)
df_films['runtime_exact'] = df_films['runtime_exact'].astype(float)

df_films['year_exact'] = df_films['year_exact'].replace('Unknown', 0)
df_films['year_exact'] = df_films['year_exact'].astype(float)

df_films['prod_US'] = df_films['prod_US'].replace('Unknown', 0)
df_films['prod_US'] = df_films['prod_US'].astype(float)

df_films['prod_FR'] = df_films['prod_FR'].replace('Unknown', 0)
df_films['prod_FR'] = df_films['prod_FR'].astype(float)

EXPORT

In [359]:
len(df_films)

27363

In [362]:
df_films.columns

Index(['title_out_KNN', 'film_id_out_KNN', 'year_exact', 'prod_FR', 'prod_US',
       'backdrop_path_out_KNN', 'homepage_out_KNN', 'overview_out_KNN',
       'poster_path_out_KNN', 'status_out_KNN',
       ...
       'debut_titre_critere_detectiveknight',
       'debut_titre_critere_acinderellastory', 'debut_titre_critere_spiderman',
       'debut_titre_critere_perrymason', 'debut_titre_critere_profilspaysans',
       'debut_titre_critere_xmen', 'debut_titre_critere_frenchtouch',
       'debut_titre_critere_transformers', 'debut_titre_critere_lexorciste',
       'arrondi_vote_exact'],
      dtype='object', length=836)

In [241]:
# colonnes = ['genre_2', 'production_companies_name_2', 'trie_genre_', 'trie_production_companies_name_']

# for element in colonnes:
#     if element in df_films.columns:
#         df_films = df_films.rename({element: f'{element}_out_KNN'}, axis = 1)

In [363]:
df_films.to_csv('P2_G5_films.csv.gz', index=False, compression='gzip')